This from FastAi chap 12

In [ ]:
from fastai.text.all import *
path = untar_data(URLs.HUMAN_NUMBERS)

In [ ]:
Path.BASE_PATH = path

In [ ]:
lines = L()
with open(path/'train.txt') as f: lines += L(*f.readlines())
with open(path/'valid.txt') as f: lines += L(*f.readlines())
lines

(#9998) ['one \n','two \n','three \n','four \n','five \n','six \n','seven \n','eight \n','nine \n','ten \n'...]

In [ ]:
tokens = " . ".join(lines).replace("\n", "").split()

In [ ]:
vocablist = L(tokens).unique()

In [ ]:
word2idx = {w:i for i,w in enumerate(vocablist)}
nums = L(word2idx[i] for i in tokens)
nums

(#63095) [0,1,2,1,3,1,4,1,5,1...]

In [ ]:
word2idx["thirty"]

21

In [ ]:
# Split into x-y groups(3 xs + 1 y)
data = [ (tokens[i:i+3], tokens[i+3]) for i in range(0,10000, 4)        ]
seqs = [ (nums[i:i+3],   nums[i+3])   for i in range(0,10000, 4)        ]

In [ ]:
seqs[1024]

((#3) [1,9,28], 15)

In [ ]:
bs = 64
cut = int(len(seqs) * 0.8)
dls = DataLoaders.from_dsets(seqs[:cut], seqs[cut:], bs=64, shuffle=False)

In [ ]:
dls.one_batch()

((#3) [tensor([ 0,  3,  5,  7,  9, 11, 13, 15, 17, 19, 20,  2,  1, 20,  6,  1, 20,  1,
        21,  3,  1, 21,  7,  1, 22,  1, 22,  4,  1, 22,  8,  1,  0,  1, 23,  5,
         1, 23,  9, 24,  2,  1, 24,  6,  1, 24,  1, 25,  3,  1, 25,  7,  1, 26,
         1, 26,  4,  1, 26,  8,  1,  0,  1, 27]),tensor([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  0,  1, 20,  5,  1, 20,  9, 21,
         2,  1, 21,  6,  1, 21,  1, 22,  3,  1, 22,  7,  1, 23,  1, 23,  4,  1,
        23,  8,  1,  0,  1, 24,  5,  1, 24,  9, 25,  2,  1, 25,  6,  1, 25,  1,
        26,  3,  1, 26,  7,  1, 27,  1, 27,  4]),tensor([ 2,  4,  6,  8, 10, 12, 14, 16, 18, 20,  1, 20,  4,  1, 20,  8,  1,  0,
         1, 21,  5,  1, 21,  9, 22,  2,  1, 22,  6,  1, 22,  1, 23,  3,  1, 23,
         7,  1, 24,  1, 24,  4,  1, 24,  8,  1,  0,  1, 25,  5,  1, 25,  9, 26,
         2,  1, 26,  6,  1, 26,  1, 27,  3,  1])],
 tensor([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1, 20,  3,  1, 20,  7,  1, 21,  1,
         21,  4,  1, 21,  8,  1,  0,  1, 22

X can be split into

  
        Embedding         Linear ReLU       Adding each ReLU together   Output
X[0]       ->                 ->                       +
X[1]       ->                 ->                       +                ->
X[2]       ->                 ->                       +

Which then eahc go through an embedding, and a Linear ReLU which adds them together


View this in edit mode to see the actual thing

In [ ]:
class RNN(Module):

    def __init__(self, vocab_size):

        hidden_size = 64
        self.input = Embedding(vocab_size, hidden_size)
        self.transform = nn.Linear(hidden_size, hidden_size)
        self.output = nn.Linear(hidden_size, vocab_size)

    def forward(self, x):
        first_word = x[0]
        second_word = x[1]
        third_word = x[2]

        h = self.input(first_word)
        h = self.transform(h)
        h = nn.ReLU()(h)

        h = h + self.input(second_word)
        h = self.transform(h)
        h = nn.ReLU()(h)

        h = h + self.input(third_word)
        h = self.transform(h)
        h = nn.ReLU()(h)

        return self.output(h)

        # def forward(self, x):
        # h = 0

        # for i in range(3):
        #     h = h + self.input(x[i])
        #     h = self.transform(h)
        #     h = nn.ReLU()(h)

        # return self.output(h)


In [ ]:
learn = Learner(dls, RNN(len(vocablist)), loss_func=F.cross_entropy, metrics=accuracy)

In [ ]:
learn.fit_one_cycle(10, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.899529,1.759627,0.444000,00:00
1,1.871449,1.608306,0.444000,00:00
2,1.826982,1.483343,0.552000,00:00
3,1.770807,1.408406,0.604000,00:00
4,1.717964,1.372753,0.602000,00:00
5,1.671117,1.357907,0.588000,00:00
6,1.631286,1.350509,0.594000,00:00
7,1.600280,1.346097,0.596000,00:00
8,1.578764,1.344082,0.598000,00:00
9,1.565579,1.343666,0.598000,00:00


In [ ]:
pd.Series(tokens).value_counts(normalize=True)

,proportion
.,0.158444
hundred,0.142642
thousand,0.142626
six,0.045962
nine,0.045962
seven,0.045962
one,0.045962
five,0.045962
four,0.045962
two,0.045962
